In [1]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")

indices<-Indices()

total output string length exceeded 10000 characters. please stay within the limit.

In [ ]:
today <- Sys.Date()
startDt <- today - 10

indices$NseTimeSeries() %>%
  filter(TIME_STAMP >= startDt) %>%
  arrange(desc(TIME_STAMP)) %>%
  print(n=Inf)

In [ ]:
indices$IndiaVixTimeSeries()

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)